In [10]:
import torch
from torch.autograd import Function
input = torch.Tensor([[4,2,3,2],
        [3,1,5,1],
        [5,6,5,3],
        [2,1,3,1]])
input = input/10
input.requires_grad=True
print(input)

tensor([[0.4000, 0.2000, 0.3000, 0.2000],
        [0.3000, 0.1000, 0.5000, 0.1000],
        [0.5000, 0.6000, 0.5000, 0.3000],
        [0.2000, 0.1000, 0.3000, 0.1000]], requires_grad=True)


In [25]:
input.transpose(1,0)

tensor([[0.4000, 0.3000, 0.5000, 0.2000],
        [0.2000, 0.1000, 0.6000, 0.1000],
        [0.3000, 0.5000, 0.5000, 0.3000],
        [0.2000, 0.1000, 0.3000, 0.1000]], grad_fn=<TransposeBackward0>)

In [11]:
from numpy.fft import rfft2, irfft2
import numpy as np
class BadFFTFunction(Function):
    @staticmethod
    def forward(ctx, input):
        numpy_input = input.detach().numpy()
        result = np.sum(numpy_input)
        result = result.squeeze()
        return torch.Tensor([result])

    @staticmethod
    def backward(ctx, grad_output):
        return result

# since this layer does not have any parameters, we can
# simply declare this as a function, rather than as an nn.Module class
def incorrect_fft(input):
    return BadFFTFunction.apply(input)

In [9]:
result = incorrect_fft(input)
loss = torch.mean(result+5)
print(result)
loss.backward()
print(input.grad)

tensor([4.7000], grad_fn=<BadFFTFunctionBackward>)


RuntimeError: Function BadFFTFunctionBackward returned an invalid gradient at index 0 - got [1] but expected shape compatible with [4, 4]

In [5]:
from numpy.fft import rfft2, irfft2

class BadFFTFunction(Function):
    @staticmethod
    def forward(ctx, input):
        numpy_input = input.detach().numpy()
        result = abs(rfft2(numpy_input))
        return input.new(result)

    @staticmethod
    def backward(ctx, grad_output):
        numpy_go = grad_output.numpy()
        result = irfft2(numpy_go)
        return grad_output.new(result)

# since this layer does not have any parameters, we can
# simply declare this as a function, rather than as an nn.Module class


def incorrect_fft(input):
    return BadFFTFunction.apply(input)
result = incorrect_fft(input)
print(result)
result.backward(torch.randn(result.size()))
print(input)

tensor([[4.7000, 0.3606, 1.3000],
        [0.8544, 0.4123, 0.3606],
        [1.3000, 0.5000, 0.5000],
        [0.8544, 0.2236, 0.3606]], grad_fn=<BadFFTFunctionBackward>)
tensor([[0.4000, 0.2000, 0.3000, 0.2000],
        [0.3000, 0.1000, 0.5000, 0.1000],
        [0.5000, 0.6000, 0.5000, 0.3000],
        [0.2000, 0.1000, 0.3000, 0.1000]], requires_grad=True)


In [6]:
loss = torch.nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
#output.backward()

In [25]:
x = torch.Tensor([[5,2],[2,3]])
y = torch.Tensor([[3,1],[1,6]])
z = torch.nn.functional.mse_loss(x,y)

In [26]:
z

tensor(3.7500)

In [17]:
import sift_pose_est
loss = torch.tensor([1.2])
loss.requires_grad = True

In [22]:
grad = torch.autograd.grad(loss, input)

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [21]:
grad

(None,)

In [27]:
import torch
from torch.autograd import Function
from numpy.fft import rfft2, irfft2


class BadFFTFunction(Function):
    @staticmethod
    def forward(ctx, input):
        numpy_input = input.detach().numpy()
        result = abs(rfft2(numpy_input))
        return input.new(result)

    @staticmethod
    def backward(ctx, grad_output):
        numpy_go = grad_output.numpy()
        result = irfft2(numpy_go)
        return grad_output.new(result)

# since this layer does not have any parameters, we can
# simply declare this as a function, rather than as an nn.Module class


def incorrect_fft(input):
    return BadFFTFunction.apply(input)

In [28]:
input = torch.randn(4, 4, requires_grad=True)
print(input)
result = incorrect_fft(input)
print(result)
result.backward(torch.randn(result.size()))
print(input)

tensor([[-0.7619,  0.0418,  0.3868, -0.7193],
        [-0.6849,  0.5129, -0.9395, -0.6971],
        [-0.1865, -0.9519,  0.2439, -0.2047],
        [ 0.0400, -0.6329, -2.8512, -0.2795]], requires_grad=True)
tensor([[7.6839, 1.7923, 1.8226],
        [1.9156, 2.5455, 1.0204],
        [3.3802, 4.7993, 4.8553],
        [1.9156, 4.2301, 1.0204]], grad_fn=<BadFFTFunctionBackward>)
tensor([[-0.7619,  0.0418,  0.3868, -0.7193],
        [-0.6849,  0.5129, -0.9395, -0.6971],
        [-0.1865, -0.9519,  0.2439, -0.2047],
        [ 0.0400, -0.6329, -2.8512, -0.2795]], requires_grad=True)
